In [ ]:
import sys
import os
from pathlib import Path

project_root = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
sys.path.insert(0, str(project_root))

from core.backtesting_opt import _Data, Strategy, Backtest
from utils.functions import BlackScholes as bs
import pandas as pd
import numpy as np
import math
from scipy.optimize import root_scalar
from datetime import datetime, date as DateObject

In [20]:
class IV_Rank(Strategy):
    upper_threshold = 0.5
    lower_threshold = 0.25
    position_id= 0
    window = 100
    
    def init(self):
        self.legs = {
            'leg1': {'type': 'CE', 'expiry_type': 'weekly', 'expiry_range': [12, 20], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None},
            'leg2': {'type': 'PE', 'expiry_type': 'weekly', 'expiry_range': [12, 20], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None},
            'leg3': {'type': 'CE', 'expiry_type': 'monthly', 'expiry_range': [26, 34], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None},
            'leg4': {'type': 'PE', 'expiry_type': 'monthly', 'expiry_range': [26, 34], 'target_strike': 'ATM', 'stop_loss': None, 'take_profit': None}
        }
        # Flag to track if we've already executed today
        self.executed_today = False
        self.last_execution_date = None
        self.previous_signal = 0
        self.iv_rank = []
    
    def execution_window(self, start_time_str="14:30:00", end_time_str="15:00:00"):
        current_time = pd.Timestamp(self.time).time()
        current_date = pd.Timestamp(self.time).date()
        
        # Reset execution flag for new trading day
        if self.last_execution_date != current_date:
            self.executed_today = False
            self.last_execution_date = current_date
        
        # Check if current time is within execution window
        start_time = pd.Timestamp(start_time_str).time()
        end_time = pd.Timestamp(end_time_str).time()
        
        if not (start_time <= current_time <= end_time):
            return False
            
        # Execute only once per day during the time window
        if self.executed_today:
            return False
            
        # Mark as executed for today
        self.executed_today = True
        return True
    
    def next(self):
        if self.spot is None or pd.isna(self.spot):
            return
            
        # Check if we should execute today
        if not self.execution_window():
            return
        
        atm = round(self.spot / 50) * 50
        
        for leg in self.legs.values():
            available_ttes = list(self.tte_to_expiry.keys()) if self.tte_to_expiry else []
            lower, upper = leg["expiry_range"]
            matching_ttes = [tte for tte in available_ttes if lower <= tte <= upper]
            if not matching_ttes:
                # print(f"No matching TTEs found for leg {leg['type']} with range {leg['expiry_range']}. Available TTEs: {sorted(available_ttes) if available_ttes else 'None'}")
                return 
            
            valid_tte = min(matching_ttes)
            leg["expiry"] = self.tte_to_expiry[valid_tte]
            
            leg["strike"] = float(atm)
            contract = f"NIFTY{pd.Timestamp(leg['expiry']).strftime('%d%b%y').upper()}{int(leg['strike'])}{leg['type']}"
            leg["contract"] = contract
            leg["data"] = self._data.get_ticker_data(contract)
            
        missing_legs = [leg["contract"] for leg in self.legs.values() if leg["data"] is None]
        if missing_legs:
            print(f"IV not found for {self.time}. Spot: {self.spot} Missing legs: {missing_legs}")
            return
        
        iv = (self.legs['leg1']['data']['iv'] + self.legs['leg2']['data']['iv'])/2
        
        if len(self.iv_rank) < self.window:
            self.iv_rank.append(iv)
            return
        
        self.iv_rank.append(iv)
        if len(self.iv_rank) > self.window:
            self.iv_rank.pop(0)
        
        iv_rank = (sum(1 for x in self.iv_rank if x <= iv) / len(self.iv_rank)) if self.iv_rank else 0
        
        print("iv_rank: ", iv_rank)
        
        
        active_trades = self.active_trades
        new_signal = 0
        if iv_rank > self.upper_threshold:
            new_signal = -1
        elif iv_rank < self.lower_threshold:
            new_signal = 1
        
        if new_signal == -1:
            self.entry_type_dict = {'weekly': 'SELL', 'monthly': 'BUY'}
        elif new_signal == 1:
            self.entry_type_dict = {'weekly': 'BUY', 'monthly': 'SELL'}
        elif new_signal == 0:
            return
            
        
        if not active_trades:
            
            placed_any_leg = False
            for leg_id, leg in self.legs.items():
                entry_type = self.entry_type_dict.get(leg["expiry_type"])
                order_fn = {'BUY': self.buy, 'SELL': self.sell}.get(entry_type)
                if order_fn is None:
                    continue
                order_fn(
                    strategy_id='strat1',
                    position_id=self.position_id,
                    leg_id=leg_id,
                    ticker=leg["contract"],
                    quantity=1,
                    stop_loss=None,
                    take_profit=None,
                    tag=f'{new_signal} signal entry'
                )
                placed_any_leg = True
            if placed_any_leg:
                self.position_id += 1
                self.previous_signal = new_signal
        else:
            near_expiry = None
            for trade in active_trades:
                expiry = datetime.strptime(trade.ticker[-14:-7], "%d%b%y").date()
                near_expiry = expiry if near_expiry is None else min(near_expiry, expiry)
            
            exit_reason = (
                "Near Expiry reached" if (pd.Timestamp(self.time).date() == near_expiry) else
                "Signal changed" if (self.previous_signal != new_signal) else
                None
            )
            
            if exit_reason:
                for trade in active_trades:
                    print("Closing position")
                    trade.close(trade.size, tag=exit_reason)
                self.previous_signal = 0
                
                placed_any_leg = False
                for leg_id, leg in self.legs.items():
                    entry_type = self.entry_type_dict.get(leg["expiry_type"])
                    order_fn = {'BUY': self.buy, 'SELL': self.sell}.get(entry_type)
                    if order_fn is None:
                        continue
                    order_fn(
                        strategy_id='strat1',
                        position_id=self.position_id,
                        leg_id=leg_id,
                        ticker=leg["contract"],
                        quantity=1,
                        stop_loss=None,
                        take_profit=None,
                        tag=f'{new_signal} signal entry'
                    )
                    placed_any_leg = True
                if placed_any_leg:
                    self.position_id += 1
                    self.previous_signal = new_signal
                
                
            else:
                if self.previous_signal == new_signal:
                    leg_strike = self.legs["leg4"]["strike"]
                    if (0.99*leg_strike) <= self.spot <= (1.01*leg_strike):
                        pass
                    else:
                            
                            # Place orders according to entry type
                            placed_any_leg = False
                            for leg_id, leg in self.legs.items():
                                entry_type = self.entry_type_dict.get(leg["expiry_type"])
                                order_fn = {'BUY': self.buy, 'SELL': self.sell}.get(entry_type)
                                if order_fn is None:
                                    continue
                                print(f"Placing {entry_type} order for {leg_id}: {leg['contract']}")
                                order_fn(
                                    strategy_id='strat1',
                                    position_id=self.position_id,
                                    leg_id=leg_id,
                                    ticker=leg["contract"],
                                    quantity=1,
                                    stop_loss=None,
                                    take_profit=None,
                                    tag=f'Adjustment {entry_type} at ATM {atm}'
                                )
                                placed_any_leg = True
                            
                            if placed_any_leg:
                                self.position_id += 1

In [ ]:
db_path = "../core/nifty_1min_desiquant.duckdb"

bt = Backtest(db_path=db_path, strategy=IV_Rank, cash=10000000, commission_per_contract=0.65, option_multiplier=75)
stats = bt.run(start_date="2021-01-01", end_date="2023-01-01")
print(stats)

In [ ]:
stats

In [ ]:
bt.tear_sheet()

In [ ]:
constraint = lambda x: x.upper_threshold > x.lower_threshold

stats, heatmap = bt.optimize(
    upper_threshold = [0.5, 0.6, 0.7, 0.8, 0.9],
    lower_threshold = [0.1, 0.2, 0.3, 0.4, 0.5],
    window = [10],
    maximize='Sharpe Ratio',
    method='grid',
    # max_tries=10,
    constraint=constraint,
    random_state=0,
    start_date="2022-01-01",
    end_date="2022-03-01",
    return_heatmap=True)

In [ ]:
stats